In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
import os
os.chdir(r'/workspaces/demo-repo/src/')

import prepare_data, train, preprocess

warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv("/workspaces/demo-repo/data/transfusion.csv")
df.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [3]:
config = prepare_data.load_config()

2023-08-24 10:30:07,074 - DEBUG - loading config.yml...


In [4]:
model = 'RandomForestClassifier'
ds_config = config['transfusion']

In [5]:
ds_config

{'name': 'transfusion',
 'input_file': {'name': 'transfusion.csv',
  'path': '/workspaces/demo-repo/data',
  'drop_nulls': ['whether he/she donated blood in March 2007']},
 'target': 'whether he/she donated blood in March 2007',
 'models': {'LogisticRegression': 'none', 'RandomForestClassifier': 'none'}}

In [6]:
if 'params' in ds_config['models']['RandomForestClassifier']:
    print(ds_config['models']['RandomForestClassifier']['params'])

In [7]:
df = prepare_data.prepare(model, ds_config)
df.head()

2023-08-24 10:30:09,995 - DEBUG - loading dataset transfusion.csv...
2023-08-24 10:30:10,017 - DEBUG - convert all column names to lowercase...
2023-08-24 10:30:10,031 - DEBUG - dropping NaN from ['whether he/she donated blood in March 2007']...


KeyError: ['whether he/she donated blood in March 2007']

In [14]:
if 'transform' in ds_config:
    df = preprocess.apply_preprocessing(df, model, ds_config)

In [16]:
train.train_and_predict(df, model, ds_config)

2023-08-24 09:51:36,495 - DEBUG - loading prepared data for training...
2023-08-24 09:51:36,498 - DEBUG - splitting train and test...
2023-08-24 09:51:36,940 - DEBUG - accuracy for RandomForestClassifier(): 0.8072916666666666
